https://forums.developer.nvidia.com/t/pytorch-for-jetson/72048

In [ ]:
from kafka import KafkaConsumer
import json
import numpy as np
import cv2
import matplotlib.pyplot as plt
import base64
import torch
from telegram import Bot
import time

KAFKA_HOST = #
TOKEN = #
CHAT_ID = #
ALARM_TIME_DIFFERENCE = 300 # 5 min = 300 sec
MAX_MESSAGE_AGE = 30

%matplotlib inline

torch.cuda.is_available()

In [ ]:
webcam_consumer = KafkaConsumer(
    bootstrap_servers=[KAFKA_HOST], 
    value_deserializer=lambda x: json.loads(x.decode('utf-8')))

webcam_consumer.subscribe(['webcam', 'webcam2', 'host_discovered'])

In [ ]:
def decode_image(image):
    image = image.encode('utf-8')
    image = base64.b64decode(image)
    image = np.frombuffer(image, dtype=np.uint8)
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    
    return image

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True)

def detect(image):
    with torch.no_grad():
        s = time.time()
        results = model([image])
        # print(time.time() - s, results)
        return results
    
def annotate(image):
    results = detect(image)
    # get persons
    persons = [ i for i in results.crop(save=False) if i['cls'] == 0 and i['conf'] > 0.7 ]
    if len(persons) == 0:
        #print('No person detected')
        return False, image
    else:
        for person in persons:
            x1 = int(person['box'][0])
            y1 = int(person['box'][1])
            x2 = int(person['box'][2])
            y2 = int(person['box'][3])
            image = cv2.rectangle(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
            
        return True, image

In [ ]:
telegram_bot = Bot(TOKEN)

async def send_alert(image):
    await telegram_bot.send_message(CHAT_ID, '<b>Person detected</b>', parse_mode='HTML')
    _, image = cv2.imencode('.jpeg', image)
    await telegram_bot.send_photo(CHAT_ID, image.tobytes())

In [ ]:
%matplotlib inline

is_at_home = False
last_alarm_timestamp = -1
is_alarm_active = False

for message in webcam_consumer:
    print(message.topic)
    current_timestamp = time.time()
    
    if current_timestamp - (message.timestamp // 1000) > MAX_MESSAGE_AGE: # ignore images older than 30 sec
        continue
    
    if message.topic == 'host_discovered':
        is_at_home = False#message.value['value']
    else:
        image = decode_image(message.value)
        if not image is None:
            detection, image = annotate(image)
            print('Is alarm active?', is_alarm_active, 'Is person detected?', detection, 'Is Hagen at home?', is_at_home, 'How long since last alarm?', current_timestamp - last_alarm_timestamp)
            
            if not detection:
                is_alarm_active = False
            
            if (not is_alarm_active) and detection and (not is_at_home) and (current_timestamp - last_alarm_timestamp > ALARM_TIME_DIFFERENCE):
                await send_alert(image)
                #plt.imshow(image)
                #plt.show()
                print('Alarm')
                last_alarm_timestamp = current_timestamp
                is_alarm_active = True